In [2]:
import pandas as pd
import os
import glob
import re

from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV

import numpy as np

import matplotlib.pyplot as plt

In [23]:
# Creación del dataframe

path = os.getcwd()
csv_files = glob.glob(os.path.join("./", "dataset/*.csv"))
#print(csv_files)

dataset = pd.DataFrame()

for f in csv_files:
    brand = re.sub(r'\.\/dataset\\', "", f)
    brand = brand.replace(".csv", "")
    
    df = pd.read_csv(f)
    df["brand"] = brand
    
    tax_c = [x for x in df.columns if re.match(r'tax', x)]
    
    if "tax" not in tax_c and tax_c != []:
        tax_c_str = tax_c[0]
        df.rename(columns = {tax_c_str : 'tax'}, inplace = True)
    
    dataset = dataset.append(df)

#Ideas
#Quitar engineSize = 0
#dataset[dataset["model"] == " C Class"]
    
dataset = dataset.fillna(0)
#Quitamos las taxes y los mpg porque 
#dataset = dataset.dropna(axis = 1)
#Quitamos los engines que tienen tamaño 0, porque no tiene sentido
dataset = dataset[dataset["engineSize"] != 0]
#Quitamos los coches que son del año 1970 y 2060, son solo dos coches --> erroneo
dataset = dataset[(dataset["year"] != 2060) & (dataset["year"] != 1970)]
max(set(list(dataset["price"])))
dataset[dataset["price"] == max(dataset["price"])]
#lm = dataset.groupby(["model"])
#dataset
#dataset
#e_size = lm.get_group(" A1")
#e_size[e_size["engineSize"] == 0]
#models = dataset["model"]
#dataset[(dataset["model"] == models) & (dataset["engineSize"] == 0)]
#df = dataset[]
#dataset = dataset[dataset["engineSize"] != 0]
#set(list(dataset["year"]))
#dataset[dataset["tax"].isnull()]

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,brand
6199,G Class,2020,159999,Semi-Auto,1350,Petrol,145.0,21.4,4.0,merc


In [3]:
bins = np.arange(0, 160000, 5000)
plt.hist(dataset['price'], bins = bins)

NameError: name 'dataset' is not defined

In [24]:
# Preprocesamiento de datos

scaler = preprocessing.MinMaxScaler()
#label_enc = preprocessing.OneHotEncoder(sparse = False) #Tiene mas sentido
label_enc = preprocessing.LabelEncoder()


# Datos a regularizar
reg_data = ['year', 'mileage', 'tax', 'mpg', 'engineSize']
#reg_data = ['year', 'mileage', 'engineSize']

# Datos para etiquetar, con OneHotEncoder
lab_data = ['model', 'transmission', 'fuelType', 'brand']

#print(dataset)

dataset[reg_data] = scaler.fit_transform(dataset[reg_data])
#dataset[reg_data] = preprocessing.normalize(dataset[reg_data], norm = 'l2')

for c in lab_data:
    dataset[c] = label_enc.fit_transform(dataset[c])

#transformed = label_enc.fit_transform(dataset[lab_data[0]].to_numpy().reshape(-1, 1))
#Create a Pandas DataFrame of the hot encoded column
#ohe_df = pd.DataFrame(transformed, columns=label_enc.get_feature_names())
#concat with original data
#dataset = pd.concat([dataset, ohe_df], axis=1).drop([c], axis=1)
#ohe_df
#d = pd.concat([dataset, ohe_df], axis = 1)
#print(ohe_df.shape)
#print(dataset.shape)
#for label in label_enc.get_feature_names():
#    dataset[label] = ohe_df[label]
#dataset
#dataset.append(ohe_df, ignore_index = True)

# FUNCIONAAAAA ******************************************************************

#for c in lab_data:
    #transformed = label_enc.fit_transform(dataset[c].to_numpy().reshape(-1, 1))
    #Create a Pandas DataFrame of the hot encoded column
    #ohe_df = pd.DataFrame(transformed, columns=label_enc.get_feature_names())
    #concat with original data
    #ohe_df = pd.get_dummies(dataset[c], prefix = c)
    
    #dataset = pd.concat([dataset, ohe_df], axis=1)
    #dataset = dataset.drop(columns = c)   

#for c in lab_data:
 #   dataset[c] = label_enc.fit_transform(dataset[c])
    
#print(dataset)

dataset = dataset.sample(frac = 1, random_state = 4815)
y = dataset['price']
X = dataset.drop(columns = 'price')



#X = dataset[['year', 'mileage', 'engineSize']]
#X

#dataset['year'] = scaler.fit_transform(dataset['year'].reshape(-1, 1))
#dataset['price'] = scaler.fit_transform(dataset['price'])
#dataset['mileage'] = scaler.fit_transorm(dataset['mileage'])
#dataset['tax'] = scaler.fit_transorm(dataset['tax'])
#dataset['mpg'] = scaler.fit_transorm(dataset['mpg'])
#dataset['engineSize'] = scaler.fit_transorm(dataset['engineSize'])

#dataset['model'] = label_enc.fit_transform(dataset['model'])
#dataset['transmission'] = label_enc.fit_transform(dataset['transmission'])
#dataset['fuelType'] = label_enc.fit_transform(dataset['fuelType'])
#dataset['brand'] = label_enc.fit_transform(dataset['brand'])

In [10]:
dataset['price'].mean()
dataset

X = X.drop(columns = 'brand')
X

,year,mileage,fuelType,tax,mpg,engineSize
17248,0.586207,0.235292,4,0.353448,0.092396,0.116667
1890,0.965517,0.010833,0,0.250000,0.120008,0.233333
4368,0.965517,0.034053,4,0.250000,0.068182,0.233333
17652,0.827586,0.024146,4,0.258621,0.101742,0.166667
13251,0.931034,0.059266,0,0.250000,0.115336,0.233333
...,...,...,...,...,...,...
5388,0.965517,0.015477,4,0.250000,0.127655,0.066667
1946,0.931034,0.036842,4,0.250000,0.127655,0.066667
2006,0.965517,0.030653,0,0.250000,0.101742,0.233333
639,0.862069,0.135394,0,0.215517,0.127655,0.233333


In [11]:
lin_reg = LinearRegression()

cross_val_score(lin_reg, X, y, cv = 5, scoring = 'neg_mean_squared_error')
#X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.3, random_state = 123)
#lin_reg.fit(X_train, Y_train)
#print(lin_reg.score(X_test, Y_test))

array([-25388026.25552295, -28118143.19578621, -24929901.67196159,
       -26765844.40334177, -27256436.33799326])

In [26]:
X = X.astype(np.float128)

poly = preprocessing.PolynomialFeatures(3)
X_poly = poly.fit_transform(X)

cross_val_score(lin_reg, X_poly, y, cv = 5, scoring = 'neg_mean_squared_error')

AttributeError: module 'numpy' has no attribute 'float128'

In [ ]:
dataset.head(5)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

parameters = {
    'n_estimators' : np.arange(100, 500),
    'criterion' : ('mse', 'mae'),
    
}

rnd_forest_reg = RandomForestRegressor()
rs = RandomizedSearchCV(rnd_forest_reg, parameters, n_iter = 25, verbose = 1, n_jobs = -1, cv = 3, random_state = 123)
search = rs.fit(X, y)

rnd_forest_reg.score(X, y)